# Hyperparameter Optimization with Optuna

This notebook uses Optuna to optimize hyperparameters for the two-stage training pipeline.

**Hyperparameters being optimized:**
- Stage 1 Learning Rate
- Stage 2 Learning Rate
- Stage 1 Epochs
- Stage 2 Epochs
- Latent Dimension (metric space dimensionality)
- Dropout rates for both models
- Batch sizes

**Optimization Metric:** Validation RMSE (lower is better)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import optuna
from optuna.trial import TrialState
import torch.nn.functional as F
import joblib
import json
from datetime import datetime

# Import our modular code
from datasets import RankedTripletDataset, RegressorFeatureDataset
from utils import load_full_dataset, fit_and_transform_pca

print("All modules imported successfully!")

All modules imported successfully!


/home/jerryjose/miniconda3/envs/ml_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Fixed configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# PCA dimensions (these remain fixed)
N_COMPONENTS_A = 87
N_COMPONENTS_BCD = 827

# Optuna configuration
N_TRIALS = 200  # Number of hyperparameter combinations to try
TIMEOUT = None  # Set to seconds if you want to limit total time

print(f"Will run {N_TRIALS} optimization trials")

Using device: cuda
Will run 200 optimization trials


## Define Configurable Model Architectures

In [3]:
class ConfigurableFinalSpecialistNN(nn.Module):
    """
    Stage 1: The 2-Tower Specialist with configurable dropout.
    """
    def __init__(self, rule_dims=87, event_dims=827, latent_dim=128, dropout1=0.2, dropout2=0.3):
        super(ConfigurableFinalSpecialistNN, self).__init__()
        self.dropout_1 = nn.Dropout(dropout1)
        self.dropout_2 = nn.Dropout(dropout2)
        
        # Tower 1: "Rule Expert" (compresses A -> latent_dim)
        self.tower1 = nn.Sequential(
            nn.Linear(rule_dims, 128),
            nn.ReLU(),
            self.dropout_1,
            nn.Linear(128, latent_dim)
        )
        
        # Tower 2: "Event Expert" (compresses BCD -> latent_dim)
        self.tower2 = nn.Sequential(
            nn.Linear(event_dims, 512),
            nn.ReLU(),
            self.dropout_2,
            nn.Linear(512, 256),
            nn.ReLU(),
            self.dropout_2,
            nn.Linear(256, latent_dim)
        )

    def forward(self, a_pca, bcd_pca):
        embed_rule = self.tower1(a_pca)
        embed_event = self.tower2(bcd_pca)
        return embed_rule, embed_event


class ConfigurableSimpleRegressorNN(nn.Module):
    """
    Stage 2: The Score Calibrator with configurable dropout.
    """
    def __init__(self, input_size=384, dropout=0.5):
        super(ConfigurableSimpleRegressorNN, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.layers = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            self.dropout,
            nn.Linear(256, 128),
            nn.ReLU(),
            self.dropout,
            nn.Linear(128, 1)
        )

    def forward(self, x):
        return self.layers(x).squeeze(-1)

print("Model architectures defined!")

Model architectures defined!


## Load and Prepare Data (Once)

In [4]:
# Load data once (will be reused across all trials)
print("Loading dataset...")
full_data = load_full_dataset('/home/jerryjose/DA5401/DA5401-data_challenge/data/train_data_embedded.json')

# Drop singleton metric_id classes
counts = full_data['metric_id'].value_counts()
singles = counts[counts == 1].index.tolist()
if singles:
    print(f"Dropping {len(singles)} singleton metric_id(s)")
    full_data = full_data[~full_data['metric_id'].isin(singles)].reset_index(drop=True)

# Train/Val split
df_train, df_val = train_test_split(
    full_data,
    test_size=0.2,
    random_state=42,
    stratify=full_data['metric_id']
)

print(f"Training set: {len(df_train)} samples")
print(f"Validation set: {len(df_val)} samples")

# PCA transformation (once)
print("Running PCA pipeline...")
result = fit_and_transform_pca(df_train, df_val, N_COMPONENTS_A, N_COMPONENTS_BCD)
a_pca_train, bcd_pca_train, a_pca_val, bcd_pca_val, pca_transformers = result

print("Data preparation complete!")

Loading dataset...
Training set: 3996 samples
Validation set: 1000 samples
Running PCA pipeline...
Starting leak-proof PCA pipeline...
Stacking raw vectors...
Training set: 3996 samples
Validation set: 1000 samples
Running PCA pipeline...
Starting leak-proof PCA pipeline...
Stacking raw vectors...
Fitting Scalers and PCA models on *training data only*...
Fitting Scalers and PCA models on *training data only*...
Transforming all data with fitted models...
Transforming all data with fitted models...
PCA transformation complete.
Data preparation complete!
PCA transformation complete.
Data preparation complete!


## Define the Training Pipeline Function

In [5]:
def train_two_stage_pipeline(trial, a_pca_train, bcd_pca_train, a_pca_val, bcd_pca_val, df_train, df_val):
    """
    Complete training pipeline for a single Optuna trial.
    Returns validation RMSE.
    """
    
    # ===== Suggest Hyperparameters =====
    latent_dim = trial.suggest_categorical('latent_dim', [64, 128, 256])
    
    # Stage 1 hyperparameters
    stage1_lr = trial.suggest_float('stage1_lr', 1e-5, 1e-3, log=True)
    stage1_batch_size = trial.suggest_categorical('stage1_batch_size', [128, 256, 512])
    stage1_epochs = trial.suggest_int('stage1_epochs', 10, 50)
    stage1_dropout1 = trial.suggest_float('stage1_dropout1', 0.1, 0.7)
    stage1_dropout2 = trial.suggest_float('stage1_dropout2', 0.1, 0.7)
    
    # Stage 2 hyperparameters
    stage2_lr = trial.suggest_float('stage2_lr', 1e-5, 1e-3, log=True)
    stage2_batch_size = trial.suggest_categorical('stage2_batch_size', [32, 64, 128])
    stage2_epochs = trial.suggest_int('stage2_epochs', 10, 50)
    stage2_dropout = trial.suggest_float('stage2_dropout', 0.3, 0.7)
    
    regressor_input_dim = latent_dim * 3
    
    print(f"\nTrial {trial.number}: latent_dim={latent_dim}, stage1_lr={stage1_lr:.2e}, stage2_lr={stage2_lr:.2e}")
    
    try:
        # ===== STAGE 1: METRIC LEARNING =====
        print("  Stage 1: Training metric learner...")
        
        # Create dataset and loader
        train_dataset_s1 = RankedTripletDataset(a_pca_train, bcd_pca_train, df_train)
        train_loader_s1 = DataLoader(
            train_dataset_s1,
            batch_size=stage1_batch_size,
            shuffle=True,
            drop_last=True
        )
        
        # Initialize model
        model_s1 = ConfigurableFinalSpecialistNN(
            rule_dims=N_COMPONENTS_A,
            event_dims=N_COMPONENTS_BCD,
            latent_dim=latent_dim,
            dropout1=stage1_dropout1,
            dropout2=stage1_dropout2
        ).to(DEVICE)
        
        criterion_ranked = nn.MarginRankingLoss(margin=0.0)
        optimizer_s1 = optim.Adam(model_s1.parameters(), lr=stage1_lr)
        
        # Training loop
        model_s1.train()
        for epoch in range(stage1_epochs):
            running_loss = 0.0
            
            for anchor, positive, neg_worse, margin in train_loader_s1:
                anchor = anchor.to(DEVICE)
                positive = positive.to(DEVICE)
                neg_worse = neg_worse.to(DEVICE)
                margin = margin.to(DEVICE)
                
                optimizer_s1.zero_grad()
                
                # Get embeddings
                anchor_embed = model_s1.tower1(anchor)
                positive_embed = model_s1.tower2(positive)
                neg_worse_embed = model_s1.tower2(neg_worse)
                
                # Calculate distances
                dist_positive = F.pairwise_distance(anchor_embed, positive_embed, p=2)
                dist_worse = F.pairwise_distance(anchor_embed, neg_worse_embed, p=2)
                
                # Loss
                target = torch.ones_like(margin)
                loss = criterion_ranked(dist_worse, dist_positive, target * margin)
                
                loss.backward()
                optimizer_s1.step()
                running_loss += loss.item()
            
            avg_loss = running_loss / len(train_loader_s1)
            
            # Report intermediate value for pruning
            trial.report(avg_loss, epoch)
            
            # Handle pruning
            if trial.should_prune():
                print(f"  Trial {trial.number} pruned at Stage 1, epoch {epoch}")
                raise optuna.TrialPruned()
        
        print(f"  Stage 1 complete: Final loss = {avg_loss:.4f}")
        
        # ===== STAGE 2: SCORE CALIBRATION =====
        print("  Stage 2: Training score calibrator...")
        
        # Create datasets
        train_dataset_s2 = RegressorFeatureDataset(
            a_pca_train, bcd_pca_train, df_train['score'].values, model_s1, DEVICE
        )
        val_dataset_s2 = RegressorFeatureDataset(
            a_pca_val, bcd_pca_val, df_val['score'].values, model_s1, DEVICE
        )
        
        train_loader_s2 = DataLoader(
            train_dataset_s2,
            batch_size=stage2_batch_size,
            shuffle=True,
            drop_last=True
        )
        val_loader_s2 = DataLoader(
            val_dataset_s2,
            batch_size=stage2_batch_size,
            shuffle=False
        )
        
        # Initialize model
        model_s2 = ConfigurableSimpleRegressorNN(
            input_size=regressor_input_dim,
            dropout=stage2_dropout
        ).to(DEVICE)
        
        criterion_reg = nn.MSELoss()
        optimizer_s2 = optim.Adam(model_s2.parameters(), lr=stage2_lr)
        
        # Training loop
        best_val_rmse = float('inf')
        
        for epoch in range(stage2_epochs):
            # Training
            model_s2.train()
            running_train_loss = 0.0
            
            for features, labels in train_loader_s2:
                features, labels = features.to(DEVICE), labels.to(DEVICE)
                
                optimizer_s2.zero_grad()
                outputs = model_s2(features)
                loss = criterion_reg(outputs, labels)
                loss.backward()
                optimizer_s2.step()
                running_train_loss += loss.item()
            
            # Validation
            model_s2.eval()
            all_preds = []
            all_true = []
            
            with torch.no_grad():
                for features, labels in val_loader_s2:
                    features, labels = features.to(DEVICE), labels.to(DEVICE)
                    outputs = model_s2(features)
                    all_preds.extend(outputs.cpu().numpy())
                    all_true.extend(labels.cpu().numpy())
            
            val_rmse = np.sqrt(mean_squared_error(all_true, all_preds))
            best_val_rmse = min(best_val_rmse, val_rmse)
            
            # Report intermediate value
            trial.report(val_rmse, stage1_epochs + epoch)
            
            # Handle pruning
            if trial.should_prune():
                print(f"  Trial {trial.number} pruned at Stage 2, epoch {epoch}")
                raise optuna.TrialPruned()
        
        print(f"  Stage 2 complete: Best Val RMSE = {best_val_rmse:.4f}")
        
        # Calculate rounded accuracy
        rounded_preds = np.round(all_preds)
        accuracy = np.mean(rounded_preds == np.array(all_true))
        
        print(f"  Final Validation Accuracy (rounded): {accuracy:.4f}")
        
        return best_val_rmse
        
    except Exception as e:
        print(f"  Trial {trial.number} failed with error: {str(e)}")
        raise e

print("Training pipeline function defined!")

Training pipeline function defined!


## Run Optuna Optimization

In [6]:
# Create objective function
def objective(trial):
    return train_two_stage_pipeline(
        trial, a_pca_train, bcd_pca_train, a_pca_val, bcd_pca_val, df_train, df_val
    )

# Create study
study = optuna.create_study(
    direction='minimize',  # Minimize validation RMSE
    study_name='two_stage_pipeline_optimization',
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)
)

print("Starting Optuna optimization...")
print(f"Running {N_TRIALS} trials")
print("=" * 80)

# Run optimization
study.optimize(objective, n_trials=N_TRIALS, timeout=TIMEOUT, show_progress_bar=True)

print("\n" + "=" * 80)
print("Optimization complete!")

[I 2025-11-17 13:43:02,533] A new study created in memory with name: two_stage_pipeline_optimization


Starting Optuna optimization...
Running 200 trials


  0%|          | 0/200 [00:00<?, ?it/s]


Trial 0: latent_dim=128, stage1_lr=7.15e-04, stage2_lr=3.91e-05
  Stage 1: Training metric learner...
  Stage 1 complete: Final loss = 0.0120
  Stage 2: Training score calibrator...
Stage 2: Pre-computing features with frozen model...
Stage 2: Feature pre-computation complete.
Stage 2: Pre-computing features with frozen model...
Stage 2: Feature pre-computation complete.
  Stage 1 complete: Final loss = 0.0120
  Stage 2: Training score calibrator...
Stage 2: Pre-computing features with frozen model...
Stage 2: Feature pre-computation complete.
Stage 2: Pre-computing features with frozen model...
Stage 2: Feature pre-computation complete.


Best trial: 0. Best value: 1.13559:   0%|          | 1/200 [00:09<32:28,  9.79s/it]

...

  Trial 197 pruned at Stage 1, epoch 10
  Trial 197 failed with error: 
[I 2025-11-17 14:03:34,963] Trial 197 pruned. 

Trial 198: latent_dim=64, stage1_lr=7.55e-04, stage2_lr=2.46e-04
  Stage 1: Training metric learner...
  Stage 1 complete: Final loss = 0.0001
  Stage 2: Training score calibrator...
Stage 2: Pre-computing features with frozen model...
Stage 2: Feature pre-computation complete.
Stage 2: Pre-computing features with frozen model...
Stage 2: Feature pre-computation complete.
  Stage 1 complete: Final loss = 0.0001
  Stage 2: Training score calibrator...
Stage 2: Pre-computing features with frozen model...
Stage 2: Feature pre-computation complete.
Stage 2: Pre-computing features with frozen model...
Stage 2: Feature pre-computation complete.


Best trial: 87. Best value: 0.887529: 100%|█████████▉| 199/200 [20:43<00:07,  7.72s/it]

  Stage 2 complete: Best Val RMSE = 0.9063
  Final Validation Accuracy (rounded): 0.5800
[I 2025-11-17 14:03:46,087] Trial 198 finished with value: 0.9063390311329186 and parameters: {'latent_dim': 64, 'stage1_lr': 0.0007549839608646044, 'stage1_batch_size': 128, 'stage1_epochs': 38, 'stage1_dropout1': 0.655017469034889, 'stage1_dropout2': 0.11622679826618681, 'stage2_lr': 0.00024614140244733155, 'stage2_batch_size': 64, 'stage2_epochs': 42, 'stage2_dropout': 0.4210378257374273}. Best is trial 87 with value: 0.8875286762820612.

Trial 199: latent_dim=64, stage1_lr=1.10e-04, stage2_lr=2.88e-04
  Stage 1: Training metric learner...


Best trial: 87. Best value: 0.887529: 100%|██████████| 200/200 [20:46<00:00,  6.23s/it]

  Trial 199 pruned at Stage 1, epoch 10
  Trial 199 failed with error: 
[I 2025-11-17 14:03:48,564] Trial 199 pruned. 

Optimization complete!


## Results Analysis

In [7]:
# Print best trial
print("\n" + "=" * 80)
print("BEST TRIAL RESULTS")
print("=" * 80)

best_trial = study.best_trial

print(f"\nBest Validation RMSE: {best_trial.value:.4f}")
print(f"\nBest Hyperparameters:")
for key, value in best_trial.params.items():
    print(f"  {key}: {value}")

# Save best parameters to JSON
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
best_params_file = f"best_hyperparameters_{timestamp}.json"

best_params = {
    'best_rmse': best_trial.value,
    'hyperparameters': best_trial.params,
    'trial_number': best_trial.number,
    'timestamp': timestamp
}

with open(best_params_file, 'w') as f:
    json.dump(best_params, f, indent=2)

print(f"\nBest hyperparameters saved to: {best_params_file}")


BEST TRIAL RESULTS

Best Validation RMSE: 0.8875

Best Hyperparameters:
  latent_dim: 128
  stage1_lr: 0.0007566667864100958
  stage1_batch_size: 256
  stage1_epochs: 40
  stage1_dropout1: 0.641243742834063
  stage1_dropout2: 0.22203393374815192
  stage2_lr: 0.000367734682650636
  stage2_batch_size: 128
  stage2_epochs: 33
  stage2_dropout: 0.43728960253816973

Best hyperparameters saved to: best_hyperparameters_20251117_140348.json


In [8]:
# Show statistics
print("\n" + "=" * 80)
print("OPTIMIZATION STATISTICS")
print("=" * 80)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print(f"\nTotal trials: {len(study.trials)}")
print(f"Completed trials: {len(complete_trials)}")
print(f"Pruned trials: {len(pruned_trials)}")

if len(complete_trials) > 0:
    # Get top 5 trials
    top_trials = sorted(complete_trials, key=lambda t: t.value)[:5]
    
    print("\nTop 5 Trials:")
    for i, trial in enumerate(top_trials, 1):
        print(f"\n  {i}. Trial #{trial.number}")
        print(f"     RMSE: {trial.value:.4f}")
        print(f"     Latent Dim: {trial.params['latent_dim']}")
        print(f"     Stage 1 LR: {trial.params['stage1_lr']:.2e}, Epochs: {trial.params['stage1_epochs']}")
        print(f"     Stage 2 LR: {trial.params['stage2_lr']:.2e}, Epochs: {trial.params['stage2_epochs']}")


OPTIMIZATION STATISTICS

Total trials: 200
Completed trials: 95
Pruned trials: 105

Top 5 Trials:

  1. Trial #87
     RMSE: 0.8875
     Latent Dim: 128
     Stage 1 LR: 7.57e-04, Epochs: 40
     Stage 2 LR: 3.68e-04, Epochs: 33

  2. Trial #129
     RMSE: 0.8914
     Latent Dim: 64
     Stage 1 LR: 9.15e-04, Epochs: 39
     Stage 2 LR: 3.10e-04, Epochs: 40

  3. Trial #42
     RMSE: 0.8921
     Latent Dim: 128
     Stage 1 LR: 8.81e-04, Epochs: 40
     Stage 2 LR: 4.47e-04, Epochs: 35

  4. Trial #135
     RMSE: 0.8929
     Latent Dim: 64
     Stage 1 LR: 6.45e-04, Epochs: 39
     Stage 2 LR: 2.08e-04, Epochs: 42

  5. Trial #193
     RMSE: 0.8932
     Latent Dim: 64
     Stage 1 LR: 8.27e-04, Epochs: 39
     Stage 2 LR: 3.81e-04, Epochs: 40


In [9]:
# Visualizations (optional, requires plotly)
try:
    from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice
    
    print("\nGenerating visualization plots...")
    
    # Optimization history
    fig1 = plot_optimization_history(study)
    fig1.show()
    
    # Parameter importances
    fig2 = plot_param_importances(study)
    fig2.show()
    
    # Slice plot
    fig3 = plot_slice(study)
    fig3.show()
    
    print("Visualizations displayed!")
    
except ImportError:
    print("\nPlotly not available. Skipping visualizations.")
    print("Install with: pip install plotly")


Generating visualization plots...


Visualizations displayed!


## Retrain with Best Hyperparameters

Once you've found the best hyperparameters, you can copy them back to your main training pipeline and train a final model with more epochs.

In [10]:
print("\n" + "=" * 80)
print("RECOMMENDED NEXT STEPS")
print("=" * 80)
print("\n1. Copy the best hyperparameters to your main training_pipeline.ipynb")
print("2. Increase epochs for final training (e.g., 100-200 epochs per stage)")
print("3. Train the final model and save it for inference")
print(f"\n4. Best hyperparameters are saved in: {best_params_file}")


RECOMMENDED NEXT STEPS

1. Copy the best hyperparameters to your main training_pipeline.ipynb
2. Increase epochs for final training (e.g., 100-200 epochs per stage)
3. Train the final model and save it for inference

4. Best hyperparameters are saved in: best_hyperparameters_20251117_140348.json
